# Produce msg to confluent-kafka topic

# Required connection configs for Kafka producer and admin
bootstrap.servers=pkc-921jm.us-east-2.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=7TNQGMFH6YFCE6DK
sasl.password=cflti9wuQtXGwWoqtvWGD+/N/yDHC0A2PUKUW5V+oWcQCkZ2TbgG+5ISIeBILymQ

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-b68508d9-92ff-426e-a6fd-b97fdc57b1e9

In [1]:
!pip install confluent-Kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 52.0 MB/s eta 0:00:00


In [2]:
from confluent_kafka import Producer
import json
import time
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Big Data Engineering Bootcamp with GCP, and Azure Cloud/Data Engineering/Data Sets/1MB/first_100_customers.csv')
df

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
...,...,...,...,...,...,...,...
94,94,Customer_94,Chennai,Maharashtra,India,2023-04-28,False
95,95,Customer_95,Mumbai,Karnataka,India,2023-09-29,True
96,96,Customer_96,Ahmedabad,Telangana,India,2023-06-22,False
97,97,Customer_97,Pune,Maharashtra,India,2023-10-31,True


In [4]:
#converting csv --> json

json_records = df.to_dict(orient='records')

json_file = 'first_100_customers.json'

with open(json_file,'w') as file:
  json.dump(json_records,file,indent=4)

print('File converted to JSON')

File converted to JSON


In [5]:
#kafka configuration for producer

conf = {
   'bootstrap.servers':'pkc-921jm.us-east-2.aws.confluent.cloud:9092',
   'security.protocol':'SASL_SSL',
   'sasl.mechanisms':'PLAIN',
   'sasl.username':'7TNQGMFH6YFCE6DK',
   'sasl.password':'cflti9wuQtXGwWoqtvWGD+/N/yDHC0A2PUKUW5V+oWcQCkZ2TbgG+5ISIeBILymQ',
   'session.timeout.ms':'45000',
   'client.id':'ccloud-python-client-b68508d9-92ff-426e-a6fd-b97fdc57b1e9'
}

producer = Producer(conf)

In [8]:
#sending 1 msg

topic = 'topic_0'

with open('first_100_customers.json','r') as file:
  cx_data = json.load(file)

value = cx_data[0]
key = value['customer_id']

print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [ ]:
type(value)

dict

In [ ]:
type(str(value).encode('utf-8'))

bytes

In [9]:
producer.produce(topic,key = str(key).encode('utf-8'), value=str(value).encode('utf-8'))

In [11]:
#sending multiple msgs

def delivery_status(err,msg):
  if (err):
    print(f'Message delivery failed:{err}')
  else:
    print(f'Message delivered to:{msg.topic} [{msg.partition()}] at offset {msg.offset()}')

for record in cx_data:
  try:
    msg_value = json.dumps(record)
    msg_key = str(record['customer_id']).encode('utf-8')

    producer.produce(topic, key=msg_key, value=msg_value, callback=delivery_status)
    producer.poll(1)

  except Exception as e:
    print(f'Error sending messages: {e}')

producer.flush()

print('Message send to kafka successfully')

Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f4e40> [0] at offset 68
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f7340> [2] at offset 62
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f7340> [0] at offset 69
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f7340> [2] at offset 63
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f51c0> [1] at offset 74
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f74c0> [2] at offset 64
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f74c0> [0] at offset 70
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f74c0> [1] at offset 75
Message delivered to:<built-in method topic of cimpl.Message object at 0x79ead72f72c0> [2] at offset 65
Message delivered to:<built-in method topic of cimpl.Message obj